## Finetune DETR model

In [ ]:
import gc
import pytorch_lightning as pl
import torch

from PIL import Image as PImage, ImageDraw as PImageDraw
from pytorch_lightning import Trainer, loggers as PLLoggers

from finetune_utils.finetune_0915 import FTUtils
from finetune_utils.Detr import Detr, DetrDataLoader

### Load HF Dataset

In [ ]:
MODEL_NAME = "microsoft/conditional-detr-resnet-50"
DATASET_NAME = "acervos-digitais/ft-0915"

In [ ]:
mDL = DetrDataLoader(DATASET_NAME, MODEL_NAME)

### Test Dataset

In [ ]:
img_id = 11
image = mDL.ds["train"][img_id]["image"]
annotations = mDL.ds["train"][img_id]["objects"]
draw = PImageDraw.Draw(image)

In [ ]:
for box,class_idx in zip(annotations["bbox"], annotations["category"]):
  x, y, w, h = tuple(box)
  x1, y1 = int(x), int(y)
  x2, y2 = int(x + w), int(y + h)

  draw.rectangle((x, y, x + w, y + h), outline="red", width=1)
  draw.text((x+2, y), FTUtils.ID2LABEL[class_idx], fill=(0,0,0))
  draw.text((x+2, y-12), FTUtils.ID2LABEL[class_idx], fill=(255,0,255))

display(image)

### Test DataLoader

In [ ]:
# ????

In [ ]:
try:
  del model
except:
  pass

gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = Detr(model_name=MODEL_NAME, dataloader=mDL, lr=1e-5, lr_backbone=1e-5, weight_decay=1e-4)

In [ ]:
cp = torch.load("lightning_logs/76+43e-1e-5lr-augm3/checkpoints/epoch=43-step=3300.ckpt")
model.load_state_dict(cp["state_dict"])

In [ ]:
mLogger = PLLoggers.TensorBoardLogger(save_dir=".", version="e2-augm3")
trainer = Trainer(accelerator="gpu", max_epochs=2, gradient_clip_val=0.1, logger=mLogger)
trainer.fit(model)

In [ ]:
print(model.detr_eval(mDL.ds["train"], min_threshold=0.3, thresholds=[]))
print(model.detr_eval(mDL.ds["test"], min_threshold=0.3, thresholds=[]))

### Save to HF Hub

In [ ]:
OUTPUT_MODEL_NAME = "acervos-digitais/conditional-detr-resnet-50-ft-0915-e256-augm3"

In [ ]:
model.model.push_to_hub(OUTPUT_MODEL_NAME)
model.processor.push_to_hub(OUTPUT_MODEL_NAME)

### Test Model

In [ ]:
import torch

from datasets import load_dataset
from os import path
from PIL import Image as PImage, ImageDraw as PImageDraw, ImageFont as PImageFont
from transformers import AutoImageProcessor, AutoModelForObjectDetection

from finetune_utils.finetune_0915 import FTUtils

MODEL_NAME = OUTPUT_MODEL_NAME

In [ ]:
ft0915_ds = load_dataset("acervos-digitais/ft-0915")

detr_processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = AutoModelForObjectDetection.from_pretrained(
  MODEL_NAME,
  id2label=FTUtils.ID2LABEL,
  label2id=FTUtils.LABEL2ID,
).to("cuda")

# TODO: fix these with static eval??

In [ ]:
print(eval_detr(model, detr_processor, list(ft0915_ds["train"]), min_threshold=0.3, thresholds=[]))
print(eval_detr(model, detr_processor, list(ft0915_ds["test"]), min_threshold=0.3, thresholds=[]))

In [ ]:
thresholds = [0.27, 0.27, 0.55]
print(eval_detr(model, detr_processor, list(ft0915_ds["train"]), min_threshold=0.15, thresholds=thresholds))
print(eval_detr(model, detr_processor, list(ft0915_ds["test"]), min_threshold=0.15, thresholds=thresholds))

In [ ]:
thresholds = [0.27, 0.27, 0.55]
print(eval_detr(model, detr_processor, list(ft0915_ds["train"]), min_threshold=0.15, thresholds=thresholds))
print(eval_detr(model, detr_processor, list(ft0915_ds["test"]), min_threshold=0.15, thresholds=thresholds))

In [ ]:
thresholds = [0.26, 0.26, 0.74]
print(eval_detr(model, detr_processor, list(ft0915_ds["train"]), min_threshold=0.15, thresholds=thresholds))
print(eval_detr(model, detr_processor, list(ft0915_ds["test"]), min_threshold=0.15, thresholds=thresholds))

## See some images

In [ ]:
for r in list(ft0915_ds["test"])[:48]:
  img = r["image"]
  iw, ih = img.size
  draw = PImageDraw.Draw(img)

  inputs = detr_processor(images=img, return_tensors="pt")
  pixel_values = inputs["pixel_values"].to("cuda")

  with torch.no_grad():
    outputs = model(pixel_values=pixel_values, pixel_mask=None)

  ppo = detr_processor.post_process_object_detection(outputs,
                                                     target_sizes=[(ih, iw)],
                                                     threshold=0.25)[0]

  labels_list = [l.item() for l in ppo["labels"]]
  scores_list = [round(s.item(),4) for s in ppo["scores"]]

  print("pred:", [(FTUtils.ID2LABEL[l],s) for l,s in zip(labels_list, scores_list)])
  print("labels:", [FTUtils.ID2LABEL[c] for c in r["objects"]["category"]])

  for l,b,s in zip(ppo["labels"], ppo["boxes"], ppo["scores"]):
    draw.rectangle(((b[0], b[1]), (b[2], b[3])), outline=(255, 0, 0), width=2)

  display(img)